In [ ]:
#Install packages needed to build model
!pip install transformers
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import math

In [ ]:
#Define model and which bert package to use
#model = SentenceTransformer('stsb-roberta-large')

#Mpnet Base faster than bert was and seems to be more accurate
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
#Import cleaned datasets
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#File paths
full = 'gdrive/My Drive/Amazon Project/df.csv'
combined = 'gdrive/My Drive/Amazon Project/df_text.csv'

#Import files as dataframes
df_full = pd.read_csv(full)
df_text = pd.read_csv(combined)

#Set item_id as index
df_full = df_full.set_index('item_id')
df_text = df_text.set_index('item_id')

In [ ]:
#Show full dataframe
df_full

,brand,bullet_point,item_name,product_type,item_keywords,domain_name,model_name
item_id,,,,,,,
B08511RJ3W,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Two Different P...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21
B08511NVLY,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Wooden Blocks T...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21
B07R7WMBZC,Amazon Brand - Solimo,"Snug fit for Oppo A3s, with perfect cutouts fo...",Amazon Brand - Solimo Designer Bear Printed Ha...,CELLULAR_PHONE_CASE,Oppo A3s Mobile back case cover transparent sl...,amazon.in,Oppo A3s
B085692143,Amazon Brand - Solimo,"Snug fit for Oppo Reno 2, with perfect cut-out...",Amazon Brand - Solimo Designer Daddy's Girl an...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Oppo Reno 2
B07TD993ZD,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Mic...,Amazon Brand - Solimo Designer Dark Flowers Ph...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Microsoft Lumia 650
...,...,...,...,...,...,...,...
B07T9TLVKD,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for App...,Amazon Brand - Solimo Designer Kitty with Glit...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Apple iPhone 8 Plus (With Logo Cut)
B07VX7D9WT,Amazon Brand - Solimo,"Snug fit for Mobile, with perfect cut-outs for...",Amazon Brand - Solimo Designer Pisces Design 3...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Realme 3 / Realme 3i
B0756Z91W8,Amazon Collection,imported,Platinum-Plated Sterling Silver Swarovski Zirc...,FINENECKLACEBRACELETANKLET,Fine Fashion,amazon.com.au,Platinum-Plated Sterling Silver Swarovski Zirc...


In [ ]:
#Show text combined dataframe
df_text

,item_name,Text
item_id,,
B00MGSFTLE,"Pinzon by Amazon 'Everyday' Spannbetttuch, Bau...",Sbana bajera ajustable Dimensiones 90 x 200 x...
B07WRK7NGQ,Klepe Men's Dark Grey Running Shoes-11 UK (45 ...,PU
B08511RJ3W,Amazon Brand - Solimo Designer Two Different P...,Snug fit for Samsung Galaxy M21 with perfect c...
B08511NVLY,Amazon Brand - Solimo Designer Wooden Blocks T...,Snug fit for Samsung Galaxy M21 with perfect c...
B078ZMFWC3,"Allegro Tea, Tea Spice Puer Yunnan Organic, 0....",Kosher
...,...,...
B07J4QYTNY,[アマゾンコレクション] Amazon Collection プラチナ鍍金スターリングシルバ...,CZ
B07QBRKSZQ,"CARE OF by PUMA Sac de sport, Bleu (blazer ble...",Amazon sest associ PUMA pour vous apporter le...
B08563Y4SJ,Amazon Brand - Solimo Designer Multicolor Mich...,Snug fit for Mobile with perfect cutouts for v...


In [ ]:
#Function that subsets the data
def subset(df, item_id):
  #Row that the product is located at
  target_row = pd.DataFrame(df.loc[[item_id]])
  
  #Actual values of product type and domain
  product_type = target_row.iloc[0]['product_type']
  domain_name = target_row.iloc[0]['domain_name']
  
  #Subset dataframe using product type and domain
  product_df = df[df['product_type']==product_type]
  subset_df = product_df[product_df['domain_name']==domain_name]

  #Drop target from the subset dataframe
  subset_df = pd.concat([subset_df, target_row])
  subset_df = subset_df.drop_duplicates(keep=False)

  #List containing the target row and subset dataframe
  subset =[target_row,subset_df]
  return subset

#For this function subset()[0] is the target row and subset()[1] is the dataframe that was created subsetting

In [ ]:
#Mini dataframe for testing
df_test = df_full.head(50)

#Testing value for item_id
test_id = 'B08511NVLY'	

In [ ]:
#Testing subset function
x_subset = subset(df_test, test_id)
x_subset_target = x_subset[0]
x_subset_df = x_subset[1]

x_subset_target
x_subset_df

,brand,bullet_point,item_name,product_type,item_keywords,domain_name,model_name
item_id,,,,,,,
B08511RJ3W,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Two Different P...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21
B07R7WMBZC,Amazon Brand - Solimo,"Snug fit for Oppo A3s, with perfect cutouts fo...",Amazon Brand - Solimo Designer Bear Printed Ha...,CELLULAR_PHONE_CASE,Oppo A3s Mobile back case cover transparent sl...,amazon.in,Oppo A3s
B085692143,Amazon Brand - Solimo,"Snug fit for Oppo Reno 2, with perfect cut-out...",Amazon Brand - Solimo Designer Daddy's Girl an...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Oppo Reno 2
B07TD993ZD,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Mic...,Amazon Brand - Solimo Designer Dark Flowers Ph...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Microsoft Lumia 650
B0853X6FLW,Amazon Brand - Solimo,"Snug fit for OnePlus 7T Pro, with perfect cut-...",Amazon Brand - Solimo Designer Leaf with Heart...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,OnePlus 7T Pro
B0853WVG7P,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy A50, with perfect ...",Amazon Brand - Solimo Designer Dark Night Park...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Samsung Galaxy A50
B07TF1DZ92,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Viv...,Amazon Brand - Solimo Designer Skeletan 3D Pri...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Vivo Y21L
B07TG4RFWP,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Viv...,Amazon Brand - Solimo Designer Line Texture on...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Vivo Y66
B07TB51XT7,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for LG K7,Amazon Brand - Solimo Designer Lotus 3D Printe...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,LG K7


In [ ]:
#Creates list of embeddings of target columns
def encode_target(target_df):
  #List of encoded target values
  encode = []
  
  #Loops through the columns in the target dataframe
  for column in range(target_df.shape[1]):

    #Actual value of each cell in the dataframe
    value = target_df.iloc[:,column][0]

    #This encodes the cell value to be used in the final function
    encode.append(model.encode(value, convert_to_tensor=True))

  #This will output the list of encoded values
  return encode


In [ ]:
#Testing encode
x_encode_target = encode_target(x_subset[0])

x_encode_target

[tensor([ 4.4958e-02, -7.2605e-02, -1.2743e-02, -2.7944e-02,  6.4829e-02,
         -2.5708e-02,  6.0899e-02, -1.8751e-02, -8.3936e-03,  1.1108e-02,
         -4.7758e-02,  5.9851e-02,  1.1072e-02,  9.2118e-02, -2.7282e-02,
          2.2485e-02,  4.6163e-03,  1.0644e-02,  8.2319e-03, -1.4406e-03,
         -3.7866e-03, -1.2740e-02,  1.3985e-02,  2.4032e-03,  1.6208e-02,
         -3.3212e-02, -1.1840e-02,  5.5698e-03, -3.3555e-02, -7.1459e-02,
          1.0464e-01,  9.0485e-03, -1.1113e-02,  2.3157e-02,  1.5125e-06,
         -1.8012e-02, -6.1137e-03, -4.0647e-02, -1.9304e-02, -1.1099e-02,
          4.3575e-02,  2.9651e-02, -2.5171e-02, -1.7347e-02, -2.8046e-02,
         -6.3564e-02,  4.8409e-02,  1.9664e-03, -2.3520e-02,  3.2646e-02,
         -1.5004e-02,  2.9021e-02,  5.0336e-02, -3.2937e-02,  4.3579e-02,
         -6.8723e-03,  5.1628e-03, -6.4171e-02,  4.9408e-02,  2.9391e-02,
          2.8302e-02, -9.6714e-03,  1.4065e-02,  6.6250e-03, -3.0824e-02,
          4.8633e-03,  2.6731e-02, -3.

In [ ]:
#Creates list of each column encoded in the full dataframe
def encode_dataframe(dataframe):
  #Turns dataframe into lists of each column
  dataframe_lists = []

  for col_index in range(dataframe.shape[1]):
    column_list = dataframe.iloc[:, col_index].values.tolist()
    dataframe_lists.append(column_list)

  #This will be the encoded lists of each column
  encoded_lists = []
  
  #Loops through each column list and appends it to the final encoded list
  for column in dataframe_lists:
    encoded_lists.append(model.encode(column, convert_to_tensor=True))
    
  #Function will return a list of each column encoded
  return encoded_lists

  #Need to check for nan value to append to list

In [ ]:
#Tests encode_dataframe function
x_encode_dataframe = encode_dataframe(x_subset[1])

x_encode_dataframe

[tensor([[ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145],
         [ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145],
         [ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145],
         ...,
         [ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145],
         [ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145],
         [ 0.0450, -0.0726, -0.0127,  ..., -0.0311, -0.0401, -0.0145]],
        device='cuda:0'),
 tensor([[ 0.0294, -0.0418, -0.0536,  ..., -0.0101, -0.0033, -0.0681],
         [ 0.0155, -0.0412, -0.0537,  ..., -0.0038,  0.0330, -0.0181],
         [ 0.0169,  0.0175, -0.0417,  ...,  0.0030,  0.0077, -0.0417],
         ...,
         [ 0.0408, -0.0162, -0.0294,  ..., -0.0479, -0.0223, -0.0382],
         [ 0.0617, -0.0567, -0.0129,  ..., -0.0017, -0.0293, -0.0355],
         [ 0.0431, -0.0542, -0.0144,  ..., -0.0254, -0.0070, -0.0079]],
        device='cuda:0'),
 tensor([[ 0.0983, -0.0804, -0.0516,  ..., -0.0412,  0.0028, -0.03

In [ ]:
#Gets cosine scores for each column
def cosine(target_encoding, df_encoding):
  #List of the column cosine scores lists
  cosine_list = []

  #Loops through the column index numbers
  for col_index in range(len(target_encoding)):
    #Actual tensor object creation
    cosine_scores = util.pytorch_cos_sim(target_encoding[col_index], df_encoding[col_index])[0]

    #Append tensor object to tensor list
    cosine_list.append(cosine_scores)

  return cosine_list


In [ ]:
#Tests cosine function
x_cosine = cosine(x_encode_target, x_encode_dataframe)

x_cosine

[tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        device='cuda:0'),
 tensor([1.0000, 0.7165, 0.6841, 0.3561, 0.7078, 0.8098, 0.3525, 0.3535, 0.3589,
         0.3440, 0.3682, 0.3566, 0.3320, 0.3613, 0.4009, 0.3618, 0.3338, 0.7114,
         0.6839, 0.7356, 0.3529, 0.3832, 0.3389, 0.8459, 0.3613, 0.6856, 0.3729,
         0.3251, 0.7488, 0.7195, 0.7598, 0.3422, 0.3898, 0.8459, 0.3764, 0.9767,
         0.1627, 0.3558, 0.7057, 0.3912, 0.3964, 0.3644, 0.3987, 0.3300],
        device='cuda:0'),
 tensor([0.8536, 0.7120, 0.7160, 0.7743, 0.7125, 0.8081, 0.7777, 0.7462, 0.7787,
         0.7732, 0.7830, 0.8621, 0.7293, 0.5359, 0.8268, 0.8009, 0.7720

In [ ]:
#Converts the tensor object into interpretable cosine values from the cosine function
def convert_numpy(cosine_list):
  #List of lists of tensor objects converted to values
  numpy_list = []
  
  #Loops through the tensor objects and converts them to a numpy value
  for obj in cosine_list:

    #Appends the numpy value to numpy list
    numpy_list.append(obj.cpu().data.numpy())
  
  #Returns the values lists
  return numpy_list

In [ ]:
#Test the convert to numpy function
x_convert_numpy = convert_numpy(x_cosine)

x_convert_numpy

[array([1.       , 1.       , 1.       , 1.       , 1.       , 1.       ,
        1.       , 1.       , 1.       , 1.       , 1.0000001, 1.0000001,
        1.0000001, 1.0000001, 1.0000001, 1.0000001, 1.0000001, 1.0000001,
        1.0000001, 1.0000001, 1.       , 1.       , 1.       , 1.       ,
        1.       , 1.       , 1.       , 1.       , 1.       , 1.       ,
        1.       , 1.       , 1.       , 1.       , 1.       , 1.       ,
        1.       , 1.       , 1.       , 1.       , 1.       , 1.       ,
        1.0000001, 1.0000001], dtype=float32),
 array([1.        , 0.7164979 , 0.68414336, 0.3560717 , 0.7078351 ,
        0.8098308 , 0.35249698, 0.353535  , 0.35890472, 0.34403214,
        0.36816585, 0.35656002, 0.33204025, 0.36126912, 0.4008796 ,
        0.36176378, 0.33375192, 0.71142036, 0.6839198 , 0.73556554,
        0.35290578, 0.38315773, 0.3388577 , 0.84594035, 0.36126912,
        0.685605  , 0.37294114, 0.32507616, 0.74884   , 0.7195183 ,
        0.7597735 , 0.34215

In [ ]:
#Function to build dataframe
def build_df(np_list, subset_df):
  #Builds dataframe using the numpy values and transpose because row/column are in the wrong order
  df_final = pd.DataFrame(np_list).transpose()

  #Renames the index and columns by the original subset dataframe
  df_final.columns = list(subset_df)
  df_final.index = list(subset_df.index.values)

  #Drops columns that match each other
  df_final = df_final.drop(['product_type', 'domain_name'], axis=1)

  #Adds mean column
  df_final = df_final.apply(pd.to_numeric)
  df_final['Mean'] = df_final.mean(axis=1)
  

  #Returns the final dataframe
  return df_final

In [ ]:
x_build_df = build_df(x_convert_numpy, x_subset_df)

x_build_df

,brand,bullet_point,item_name,item_keywords,model_name,Mean
B08511RJ3W,1.0,1.000000,0.853647,1.000000,1.000000,0.970730
B07R7WMBZC,1.0,0.716498,0.712041,0.528872,0.458268,0.683136
B085692143,1.0,0.684143,0.716042,0.379171,0.358687,0.627609
B07TD993ZD,1.0,0.356072,0.774309,0.539092,0.699309,0.673756
B0853X6FLW,1.0,0.707835,0.712507,0.379171,0.517238,0.663350
B0853WVG7P,1.0,0.809831,0.808089,0.379171,0.772035,0.753825
B07TF1DZ92,1.0,0.352497,0.777740,0.539092,0.504977,0.634861
B07TG4RFWP,1.0,0.353535,0.746231,0.539092,0.563797,0.640531
B07TB51XT7,1.0,0.358905,0.778679,0.539092,0.580897,0.651515
B07TF1G77C,1.0,0.344032,0.773245,0.539092,0.645891,0.660452


In [ ]:
#Uses the subset dataframe and sorts it using the mean cosine values calculated from bert
def sorted_dataframe(df_sub, df_score):
  df_sub['Mean'] = df_score['Mean']
  df_sub = df_sub.sort_values(by = ['Mean'], ascending = False)
  return df_sub

In [ ]:
x_sorted_df = sorted_dataframe(x_subset_df, x_build_df)

x_sorted_df

,brand,bullet_point,item_name,product_type,item_keywords,domain_name,model_name,Mean
item_id,,,,,,,,
B08511RJ3W,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Two Different P...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21,0.970730
B07XYDB1Z2,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21 / M30s, with p...",Amazon Brand - Solimo Designer Girl Boss On Pi...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M21 / M30s,0.847129
B081HMFQC5,Amazon Brand - Solimo,"Snug fit for Mobile, with perfect cut-outs for...",Amazon Brand - Solimo Designer Button Jeans Al...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M21 / M30s,0.816773
B0853WVG7P,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy A50, with perfect ...",Amazon Brand - Solimo Designer Dark Night Park...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Samsung Galaxy A50,0.753825
B081HNR9VG,Amazon Brand - Solimo,"Snug fit for Mobile, with perfect cut-outs for...",Amazon Brand - Solimo Designer Heart Pattern A...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,OnePlus 7T,0.746406
B0853WG1ML,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy A6 Plus, with perf...",Amazon Brand - Solimo Designer Autumn Girl 3D ...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Samsung Galaxy A6 Plus,0.725847
B07SZSTRTX,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Sam...,Amazon Brand - Solimo Designer Texture 3D Prin...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M40,0.716644
B07TBV4RNY,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Sam...,Amazon Brand - Solimo Designer Wooden Pattern ...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy J4 Plus,0.702493
B07THC8T15,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Son...,Amazon Brand - Solimo Designer Plastic Paint 3...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Sony Xperia L1,0.696055


In [ ]:
def product_recommendation(initial_df, item_id):
  #Use subset function to create a new dataframe that is split by product type and domain
  subset_final = subset(initial_df,item_id)

  #Create variables for the target and subset dataframe
  target = subset_final[0]
  df = subset_final[1]

  #Encodes the target and dataframe using the two functions encode_target and encode_dataframe
  encoded_target = encode_target(target)
  encoded_df = encode_dataframe(df)
  
  #Creates tensor objects for cosine scoring
  cosine_scores = cosine(encoded_target, encoded_df)

  #Returns cosine score values of objects
  value = convert_numpy(cosine_scores)

  #Builds score dataframe for results with cosine values
  score_df = build_df(value, df)

  #Adds mean cosine values to the dataframe and sorts using that
  sorted = sorted_dataframe(df, score_df)

  return([score_df, sorted, target])

In [ ]:
#Tests final product_recommendation function
x_recommendation = product_recommendation(df_test,test_id)


In [ ]:
x_recommendation[2]

,brand,bullet_point,item_name,product_type,item_keywords,domain_name,model_name
item_id,,,,,,,
B08511NVLY,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Wooden Blocks T...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21


In [ ]:
x_recommendation[1]

,brand,bullet_point,item_name,product_type,item_keywords,domain_name,model_name,Mean
item_id,,,,,,,,
B08511RJ3W,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21, with perfect ...",Amazon Brand - Solimo Designer Two Different P...,CELLULAR_PHONE_CASE,Samsung Galaxy M21 mobile case,amazon.in,Samsung Galaxy M21,0.970730
B07XYDB1Z2,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy M21 / M30s, with p...",Amazon Brand - Solimo Designer Girl Boss On Pi...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M21 / M30s,0.847129
B081HMFQC5,Amazon Brand - Solimo,"Snug fit for Mobile, with perfect cut-outs for...",Amazon Brand - Solimo Designer Button Jeans Al...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M21 / M30s,0.816773
B0853WVG7P,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy A50, with perfect ...",Amazon Brand - Solimo Designer Dark Night Park...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Samsung Galaxy A50,0.753825
B081HNR9VG,Amazon Brand - Solimo,"Snug fit for Mobile, with perfect cut-outs for...",Amazon Brand - Solimo Designer Heart Pattern A...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,OnePlus 7T,0.746406
B0853WG1ML,Amazon Brand - Solimo,"Snug fit for Samsung Galaxy A6 Plus, with perf...",Amazon Brand - Solimo Designer Autumn Girl 3D ...,CELLULAR_PHONE_CASE,cellphonecover,amazon.in,Samsung Galaxy A6 Plus,0.725847
B07SZSTRTX,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Sam...,Amazon Brand - Solimo Designer Texture 3D Prin...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy M40,0.716644
B07TBV4RNY,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Sam...,Amazon Brand - Solimo Designer Wooden Pattern ...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Samsung Galaxy J4 Plus,0.702493
B07THC8T15,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for Son...,Amazon Brand - Solimo Designer Plastic Paint 3...,CELLULAR_PHONE_CASE,mobile cover,amazon.in,Sony Xperia L1,0.696055


In [ ]:
x_recommendation[0]

,brand,bullet_point,item_name,item_keywords,model_name,Mean
B08511RJ3W,1.0,1.000000,0.853647,1.000000,1.000000,0.970730
B07R7WMBZC,1.0,0.716498,0.712041,0.528872,0.458268,0.683136
B085692143,1.0,0.684143,0.716042,0.379171,0.358687,0.627609
B07TD993ZD,1.0,0.356072,0.774309,0.539092,0.699309,0.673756
B0853X6FLW,1.0,0.707835,0.712507,0.379171,0.517238,0.663350
B0853WVG7P,1.0,0.809831,0.808089,0.379171,0.772035,0.753825
B07TF1DZ92,1.0,0.352497,0.777740,0.539092,0.504977,0.634861
B07TG4RFWP,1.0,0.353535,0.746231,0.539092,0.563797,0.640531
B07TB51XT7,1.0,0.358905,0.778679,0.539092,0.580897,0.651515
B07TF1G77C,1.0,0.344032,0.773245,0.539092,0.645891,0.660452


In [ ]:

from sentence_transformers import losses
train_loss = losses.CosineSimilarityLoss(model=model)
print(train_loss)

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)
